# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/My Drive/GIZ Zindi/Train.csv' .
!cp '/content/drive/My Drive/GIZ Zindi/SampleSubmission.csv' .

In [3]:
!cp '/content/drive/My Drive/GIZ Zindi/AdditionalUtterances.zip' AdditionalUtterances.zip

In [4]:
!unzip -q AdditionalUtterances.zip

In [5]:
# Copy the files in and unzip
!cp '/content/drive/My Drive/GIZ Zindi/audio_files.zip' audio_files.zip
!unzip -q audio_files.zip

In [6]:
!cp "/content/drive/My Drive/GIZ Zindi/nlp_keywords_29Oct2020.zip" nlp_keywords_29Oct2020.zip
!unzip -q nlp_keywords_29Oct2020.zip

In [7]:
!pip -q install efficientnet_pytorch

In [8]:
!pip install -q python_speech_features

In [9]:
!pip -q install albumentations --upgrade

     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 952kB 11.5MB/s 
     |████████████████████████████████| 36.7MB 87kB/s 


In [10]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.notebook import tqdm as tqdm 
from sklearn.model_selection import train_test_split
import librosa
import librosa.display as display
import python_speech_features as psf
from matplotlib import pyplot as plt
import numpy as np
import albumentations
from torch.nn import Module,Sequential
import gc
import cv2
import multiprocessing as mp 
from multiprocessing import Pool
from albumentations.augmentations.transforms import Lambda
import IPython.display as ipd 

In [11]:
N_WORKERS = mp.cpu_count()
LOAD_TRAIN_DATA = None
LOAD_TEST_DATA = None

In [12]:
import random
import numpy as np
SEED_VAL  = 1000
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED = SEED_VAL):
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)
  os.environ['PYTHONHASHSEED'] = str(SEED)
  torch.backends.cudnn.deterministic = True

# DataLoader

In [13]:
class conf:
    sampling_rate = 44100
    duration = 3 # sec
    hop_length = 200*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    padmode = 'constant'
    samples = sampling_rate * duration
def get_default_conf():
    return conf
conf = get_default_conf()

In [14]:
def melspectogram_dB(file_path, cst=3, top_db=80.):
  row_sound, sr = librosa.load(file_path,sr=conf.sampling_rate)
  sound = np.zeros((cst*sr,))
  if row_sound.shape[0] < cst*sr:
    sound[:row_sound.shape[0]] = row_sound[:]
  else:
    sound[:] = row_sound[:cst*sr]

  spec = librosa.feature.melspectrogram(sound, 
                                    sr=conf.sampling_rate,
                                    n_mels=conf.n_mels,
                                    hop_length=conf.hop_length,
                                    n_fft=conf.n_fft,
                                    fmin=conf.fmin,
                                    fmax=conf.fmax)
  spec_db = librosa.power_to_db(spec)
  spec_db = spec_db.astype(np.float32)

  return spec_db

def spec_to_image(spec, eps=1e-6):
  mean = spec.mean()
  std = spec.std()
  spec_norm = (spec - mean) / (std + eps)
  spec_min, spec_max = spec_norm.min(), spec_norm.max()
  spec_img = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
  
  return spec_img.astype(np.uint8)
def preprocess_audio(audio_path):
  spec = melspectogram_dB(audio_path)
  spec = spec_to_image(spec)
  return spec

In [15]:
def get_data(df,mode='train'):
  """
  :param: df: dataframe of train or test
  :return: images_list: spec images of all the data
  :return: label_list : label list of all the data
  """
  audio_paths = df.fn.values
  images_list = []
  with mp.Pool(N_WORKERS) as pool:
    images_list = pool.map(preprocess_audio,tqdm(audio_paths))

  if mode == 'train':
    label_list = df.label.values
    return images_list,label_list
  else:
    return images_list

In [16]:
class ImageDataset(Dataset):
    def __init__(self, images_list,labels_list=None,transform=None):
        self.images_list = images_list
        self.transform = transform
        self.labels_list = labels_list

    def __getitem__(self, index):
      
      spec = self.images_list[index]
      if self.transform is not None:
        spec = self.transform(image=spec)
        spec = spec['image']
        
      if self.labels_list is not None:
          label = self.labels_list[index]
          return {'image' : torch.tensor(spec,dtype=torch.float), 
              'label' : torch.tensor(label,dtype = torch.long) }
          
      return {'image' : torch.tensor(spec,dtype=torch.float), }

    def __len__(self):
        return len(self.images_list)

# Models and train functions

In [17]:
class Net(nn.Module):
    def __init__(self,name):
        super(Net, self).__init__()
        self.name  = name
        #self.convert_3_channels = nn.Conv2d(1,3,2,padding=1)
        if name == 'b0':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b0')
          self.arch._fc = nn.Linear(in_features=1280, out_features=193, bias=True)
        elif name == 'b1':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b1')
          self.arch._fc = nn.Linear(in_features=1280, out_features=193, bias=True)
        elif name == 'b2':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b2')
          self.arch._fc = nn.Linear(in_features=1408, out_features=193, bias=True)
        elif name =='b3':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b3')
          self.arch._fc = nn.Linear(in_features=1536, out_features=193, bias=True)
        elif name =='b4':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b4')
          self.arch._fc = nn.Linear(in_features=1792, out_features=193, bias=True,)
        elif name =='b5':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b5')
          self.arch._fc = nn.Linear(in_features=2048, out_features=193, bias=True)
        elif name =='b6':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b6')
          self.arch._fc = nn.Linear(in_features=2304, out_features=193, bias=True)
        elif name =='b7':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b7')
          self.arch._fc = nn.Linear(in_features=2560, out_features=193, bias=True)
        elif name == 'densenet121':
          self.arch = models.densenet121(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'densenet169':
          self.arch = models.densenet169(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'densenet201':
          self.arch = models.densenet201(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet50':
          self.arch = models.resnet50(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet101':
          self.arch = models.resnet101(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet152':
          self.arch = models.resnet152(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet18':
          self.arch = models.resnet18(pretrained=True)
          my_weight = self.arch.conv1.weight.mean(dim=1, keepdim=True)    
          self.arch.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          self.arch.conv1.weight = torch.nn.Parameter(my_weight)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnet34':
          self.arch = models.resnet34(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnext101':
          self.arch = models.resnext101_32x8d(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name == 'resnext50':
          self.arch = models.resnext50_32x4d(pretrained=True)
          my_weight = self.arch.conv1.weight.mean(dim=1, keepdim=True)    
          self.arch.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          self.arch.conv1.weight = torch.nn.Parameter(my_weight)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,193,bias=True)
        elif name =='rexnetv1':
            model = rexnetv1.ReXNetV1(width_mult=1.0)
            model.output.conv2D = nn.Conv2d(1280, 1, kernel_size=(1, 1), stride=(1, 1))
          
    def forward(self, x):
        """
        """
        #x = self.convert_3_channels(x)
        x = self.arch(x)
        return x

In [22]:
class AudioCrop:
    def __init__(self,percentage=0.75):
        self.percentage=percentage

    def __call__(self,image,**kwargs):
        #print("image hey : ",image.shape)
        perc=np.random.random()*(1-self.percentage)+self.percentage
        return albumentations.RandomCrop(image.shape[0],int(image.shape[1]*perc),p=1)(image=image)["image"]

        
def spec_augment(spec: np.ndarray,
                 num_mask=2,
                 freq_masking=0.15,
                 time_masking=0.20,
                 value=0):
    spec = spec.copy()
    num_mask = random.randint(1, num_mask)
    for i in range(num_mask):
        all_freqs_num, all_frames_num  = spec.shape
        freq_percentage = random.uniform(0.0, freq_masking)

        num_freqs_to_mask = int(freq_percentage * all_freqs_num)
        f0 = np.random.uniform(low=0.0, high=all_freqs_num - num_freqs_to_mask)
        f0 = int(f0)
        spec[f0:f0 + num_freqs_to_mask, :] = value

        time_percentage = random.uniform(0.0, time_masking)

        num_frames_to_mask = int(time_percentage * all_frames_num)
        t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
        t0 = int(t0)
        spec[:, t0:t0 + num_frames_to_mask] = value
    return spec

class SpecAugment:
    def __init__(self,
                 num_mask=2,
                 freq_masking=0.15,
                 time_masking=0.20):
        self.num_mask = num_mask
        self.freq_masking = freq_masking
        self.time_masking = time_masking

    def __call__(self, image,**kwargs):
        return spec_augment(image,
                            self.num_mask,
                            self.freq_masking,
                            self.time_masking,
                            image.min())

# Predicting

In [119]:
HEIGHT = 128
WIDTH = 512
def get_transforms(): 
  train_transform = albumentations.Compose([
      #albumentations.PadIfNeeded(HEIGHT,WIDTH,border_mode = cv2.BORDER_CONSTANT,value=0),
      albumentations.Resize(HEIGHT,WIDTH),
      #albumentations.Lambda(NM(),always_apply=True)
      #Lambda(image=SpecAugment(num_mask=2,freq_masking=0.1,time_masking=0.1),mask=None,p=0.2),
      #Lambda(image=GaussNoise(2),mask=None,p=0.2),
      #albumentations.Lambda(image=CONVERTRGB(),always_apply=True),
      #albumentations.CenterCrop(100,140,p=1)
      #albumentations.RandomCrop(120,120)
      #albumentations.VerticalFlip(p=0.2),
      #albumentations.HorizontalFlip(p=0.2),
      #albumentations.RandomContrast(p=0.2),
      
      #AT.ToTensor()
      ])
  val_transform = albumentations.Compose([
                              
      
      albumentations.OneOf([
        albumentations.Lambda(AudioCrop(percentage=0.95), p=0.3),
        albumentations.Lambda(AudioCrop(percentage=0.85), p=0.4),
        albumentations.Lambda(AudioCrop(percentage=0.9), p=0.6),
      ],p=0.35),

      albumentations.OneOf([
        albumentations.Lambda(SpecAugment(num_mask=1,freq_masking=0.1,time_masking=0.2)),
        albumentations.Lambda(SpecAugment(num_mask=2,freq_masking=0.1,time_masking=0.1)),
        albumentations.Lambda(SpecAugment(num_mask=3,freq_masking=0.05,time_masking=0.04)),
        albumentations.Lambda(SpecAugment(num_mask=1,freq_masking=0.05,time_masking=0.1)),
      ],p=0.25),


      
      albumentations.Resize(HEIGHT,WIDTH),
      ])
  return train_transform,val_transform

In [19]:
%%time
if LOAD_TEST_DATA is None:
  gc.collect()
  test = pd.read_csv('SampleSubmission.csv')
  #takes 5 minutes
  test_images = get_data(test,mode='test')
  LOAD_TEST_DATA = True
else:
  print('Data Already Loaded')


CPU times: user 257 ms, sys: 89.4 ms, total: 347 ms
Wall time: 3min 32s


In [120]:
_,test_transform = get_transforms()

In [121]:
test_dataset = ImageDataset(test_images,labels_list=None,transform=test_transform)
test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=32)

## KFOLDS

In [30]:
#!cp -r "/content/drive/MyDrive/resnext50_28_11" "."

In [122]:
NUM_TTAS = 15
NFOLDS = 10
NAME = 'resnext50'
all_outputs = []
device = torch.device("cuda")
final_outputs = []
for i in range(NFOLDS):
  best_model = Net(NAME)
  #best_model.load_state_dict(torch.load(f'resnext50_28_11/best_model_{i}'))
  best_model.load_state_dict(torch.load(f'best_model_{i}'))
  best_model = best_model.to(device)
  best_model.eval()
  with torch.no_grad():
    tk0 = tqdm(test_data_loader, total=len(test_data_loader))
    for tta_num in range(NUM_TTAS):
      seed_all(tta_num)
      round_preds = []
      for bi,d in enumerate(tk0):
        images = d['image']
        #send them to device 
        images = images.to(device,dtype=torch.float)
        outputs  = best_model(images.unsqueeze(dim=1))
        outputs = torch.nn.functional.softmax(outputs)
        round_preds.extend(outputs.cpu().numpy())
      final_outputs.append(round_preds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [124]:
len(final_outputs)

150

In [125]:
import scipy 
from scipy.stats.mstats import gmean

In [137]:
ss1 = pd.read_csv('/content/SampleSubmission.csv')
ss1.iloc[:,1:] = gmean(final_outputs,axis=0)
ss1.to_csv(f'resnext50_tta.csv',index=False)
ss1.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,...,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,2.343707e-04,2.167347e-06,9.917329e-06,1.732347e-06,5.240366e-07,3.096566e-06,4.434194e-04,7.849318e-04,5.572421e-04,1.559868e-07,2.600972e-05,1.144912e-06,1.224174e-07,1.613131e-06,3.763497e-06,1.891534e-04,5.226647e-03,2.051819e-06,6.057225e-05,2.718343e-06,1.615919e-06,1.056904e-02,1.596309e-06,1.048316e-05,1.696383e-06,1.059257e-05,7.992502e-07,2.865023e-05,2.935979e-07,4.614795e-02,5.134823e-04,2.960034e-03,4.251710e-07,1.168507e-05,1.070869e-05,5.832870e-05,6.651165e-05,7.369058e-07,1.248388e-03,...,8.586778e-06,3.959572e-07,6.861133e-06,2.824899e-03,1.575203e-05,2.658377e-05,4.673659e-08,2.992440e-04,2.036365e-06,3.437708e-06,1.993348e-07,0.000001,7.673174e-09,1.002542e-06,6.713438e-07,7.658627e-05,3.743121e-05,3.543033e-06,3.398936e-05,5.519286e-07,1.294995e-06,9.250061e-06,1.465595e-06,1.391163e-06,1.012797e-06,1.936667e-03,7.485040e-05,2.332874e-04,5.374549e-08,4.612908e-05,3.338487e-06,4.635766e-05,5.561492e-05,1.859332e-04,4.052986e-06,1.620548e-05,4.788840e-05,6.891407e-05,2.209776e-06,2.643778e-06
1,audio_files/00P0NMV.wav,5.638194e-09,5.332678e-10,7.372097e-10,3.970086e-08,1.005301e-08,8.887692e-08,2.056218e-09,1.739805e-07,5.615364e-10,8.397910e-08,1.293241e-07,7.838212e-09,1.004437e-07,1.800764e-08,7.815673e-08,3.515090e-07,5.563740e-11,1.135382e-06,6.501561e-10,1.087006e-08,1.690859e-10,1.625241e-07,9.359348e-06,4.530443e-09,6.150617e-09,5.872658e-08,1.207974e-09,3.432685e-08,3.138164e-09,4.953028e-10,1.258190e-09,4.501675e-12,5.630277e-08,1.242797e-07,4.211057e-11,5.341163e-08,4.545232e-10,2.787055e-09,1.132171e-07,...,2.230255e-08,4.661918e-07,1.733118e-07,1.053502e-09,7.383560e-06,1.508822e-07,8.974141e-06,1.001030e-08,5.697262e-09,3.189098e-08,1.008070e-07,0.000030,3.640287e-08,1.610718e-07,7.796359e-07,7.162465e-07,2.921623e-12,8.885640e-08,1.197035e-05,2.410994e-06,2.690062e-07,1.031602e-07,5.027077e-09,3.274041e-05,1.785209e-07,5.735961e-08,1.322030e-05,5.505606e-11,2.298041e-09,5.695966e-10,4.125670e-08,1.680026e-09,7.239609e-10,8.834407e-07,9.904966e-01,8.585365e-07,4.129028e-06,2.666985e-05,3.851477e-07,1.450141e-07
2,audio_files/01QEEZI.wav,1.749539e-07,3.782902e-07,2.399082e-07,4.004459e-05,1.105588e-06,2.435132e-06,7.814581e-07,3.024316e-07,1.959788e-08,1.603023e-04,1.371523e-05,3.994032e-07,2.759388e-04,4.003489e-07,9.476114e-06,1.397787e-07,3.368995e-08,4.140591e-01,8.229211e-07,2.271972e-06,2.583346e-07,7.486096e-09,5.379652e-06,2.152004e-07,2.206612e-07,2.248244e-07,3.595861e-07,5.151931e-08,5.667884e-07,6.240698e-09,3.223404e-09,4.424229e-09,1.596867e-07,1.773437e-06,2.437069e-07,8.973195e-07,1.209627e-06,1.784949e-07,8.422967e-08,...,1.164354e-06,1.140762e-04,7.502156e-07,4.546604e-09,9.027920e-06,2.804158e-07,2.536064e-06,3.636774e-09,8.613309e-08,6.271287e-07,1.056788e-06,0.000006,2.272516e-06,5.293956e-03,2.024907e-07,5.069633e-07,1.042512e-09,2.664960e-04,1.559653e-06,2.807679e-06,4.786277e-06,7.108058e-08,1.384535e-05,3.572391e-06,9.303773e-04,1.190306e-07,7.833612e-06,1.233943e-08,4.295438e-06,5.161935e-08,1.151988e-06,3.757961e-09,4.686690e-09,2.709770e-07,7.196669e-06,3.507120e-07,7.025515e-08,9.395383e-07,2.877179e-06,1.170449e-05
3,audio_files/037YAED.wav,1.838726e-05,1.571432e-07,2.891713e-07,2.306506e-07,1.335204e-07,2.192871e-07,5.297198